In [1]:
import gensim
from gensim.models import Word2Vec,KeyedVectors
import numpy as np

In [2]:
import gensim.downloader as api
# Load the pre-trained Word2Vec model
model = api.load("word2vec-google-news-300")
# Example words
words = ["king", "queen"]
# Get the vector for each word
vectors = [model[word] for word in words]

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [4]:
a=np.dot(vectors[0], vectors[1])  # Compute the dot product
# Compute the cosine similarity
print(a/(np.linalg.norm(vectors[0]) * np.linalg.norm(vectors[1])))

0.6510957


In [5]:
import pandas as pd
corpus= pd.read_json("News_Category_Dataset_v3.json", lines=True)
corpus.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [18]:
from nltk.corpus import stopwords
import nltk
#cleaning the text data of punctuation, make everything lowercase, removing stopwords
filtered=corpus['headline'].str.lower().str.replace('[^a-z ]', '', regex=True)
filtered = filtered.apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords.words('english')]))
filtered.head()

0    million americans roll sleeves omicrontargeted...
1    american airlines flyer charged banned life pu...
2                  funniest tweets cats dogs week sept
3                    funniest tweets parents week sept
4    woman called cops black birdwatcher loses laws...
Name: headline, dtype: object

In [19]:
sentence_vectors=[]
for sentence in filtered:
    words = sentence.split()
    vectors = [model[word] for word in words if word in model]
    if vectors:
        sentence_vector = np.mean(vectors, axis=0)
        sentence_vectors.append(sentence_vector)
    else:
        sentence_vectors.append(np.zeros(model.vector_size))  # Handle empty sentences

In [20]:
sentence_vectors[5]

array([-0.02609689,  0.0816476 ,  0.12443324, -0.06874303, -0.02518354,
       -0.00381034, -0.00080218, -0.17884691,  0.13344029,  0.16322981,
        0.1086077 , -0.19949777, -0.04643467,  0.05913435,  0.01211984,
       -0.08632115,  0.020595  ,  0.02985491,  0.0411584 ,  0.01653181,
        0.04657855, -0.03735352, -0.02765764,  0.00843157, -0.07263619,
       -0.05360631, -0.08433314,  0.00263323,  0.15037973, -0.1233575 ,
       -0.00977434,  0.02690561,  0.05687604, -0.01420593, -0.02626256,
       -0.08790807,  0.06533595,  0.04438128, -0.09943499,  0.1926967 ,
       -0.04513114, -0.0836683 ,  0.06937081,  0.04549735, -0.01101576,
       -0.08265904, -0.10972377,  0.03749302, -0.06777518,  0.05221122,
        0.04017857, -0.07247489,  0.01252093, -0.03948103, -0.02784947,
       -0.10801478, -0.03712681, -0.05824498,  0.00401524, -0.02008929,
        0.01405552,  0.2094029 , -0.10889816, -0.02481515, -0.03007398,
        0.00983102,  0.04385376,  0.05056327, -0.06953593,  0.02

In [21]:
np.random.seed(42)

In [33]:
selected_sentence= np.random.randint(0, len(sentence_vectors))
print("Selected sentence vector:", corpus['headline'][selected_sentence])
print("Cleaned sentence vector:", filtered[selected_sentence])

Selected sentence vector: A Journey From Unlovable to Beloved: An ex-Jehovah's Witness Reclaims Her Spiritual Heritage
Cleaned sentence vector: journey unlovable beloved exjehovahs witness reclaims spiritual heritage


In [34]:
#match the selected sentence vector with the corpus
#ignore the selected sentence itself
threshold=0.6
similarities = []
for i, vector in enumerate(sentence_vectors):
    if i != selected_sentence:
        similarity = np.dot(sentence_vectors[selected_sentence], vector) / (
            np.linalg.norm(sentence_vectors[selected_sentence]) * np.linalg.norm(vector)+0.01)
        if similarity > threshold:
            similarities.append((i, similarity))

In [35]:
#print similarities
print(f"Selected sentence: {corpus['headline'][selected_sentence]}")
similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
for i, sim in similarities[:10]:
    print(f"Index: {i}, Similarity: {sim:.4f}, Headline: {corpus['headline'][i]}")
    

Selected sentence: A Journey From Unlovable to Beloved: An ex-Jehovah's Witness Reclaims Her Spiritual Heritage
Index: 197021, Similarity: 0.6748, Headline: A Lost Suitcase and a Spiritual Journey
Index: 206339, Similarity: 0.6698, Headline: Reverence Thyself First: Where the Sacred Journey Begins
Index: 172774, Similarity: 0.6598, Headline: The Journey of Sacred Motherhood
Index: 161324, Similarity: 0.6595, Headline: The Journey to True Love
Index: 89366, Similarity: 0.6576, Headline: Zachary Quinto Has A 'Spiritual' Connection To 'Star Trek' Forefather Leonard Nimoy
Index: 136856, Similarity: 0.6545, Headline: My Spiritual Travels in Jerusalem
Index: 191936, Similarity: 0.6326, Headline: The Untethered Soul: Twelve-Step Guide to Spiritual Awakening
Index: 171237, Similarity: 0.6283, Headline: Why the World Needs More Spiritual Heroes
Index: 110716, Similarity: 0.6254, Headline: Bully for Ancestry: My Journey Into the Past Uncovered a Forgotten Polish Odyssey
Index: 142696, Similarity

##### We can see that most of the similarity comaparisons match sentences with common words, and not much else